In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn import metrics
from tqdm.auto import tqdm
import numpy as np
import random

import mimic3models.metrics as m
import matplotlib.pyplot as plt
import glob

from DataLoader import LoadDataSets
from lstm_cnn import LSTM_CNN4
from lstm_cnn import trainer, evaluate, calcMetrics, plotLoss, plotAUC

# Set random seeds

In [ ]:
#CUDA RNN and LSTM
#In some versions of CUDA, RNNs and LSTM networks may have non-deterministic behavior. See torch.nn.RNN() and torch.nn.LSTM() for details and workarounds.
# https://pytorch.org/docs/stable/notes/randomness.html

def randseed(seed=42):
    torch.manual_seed(seed)
    random.seed(seed)
    np.random.seed(seed)

randseed()

# Load Training, Test and Validation Data Sets

Set `mimic4` to `True` if you want to evaluate against MIMIC-IV, or to `False` for MIMIC-III.

In [ ]:
try:
    del train_data
except:
    pass

try:
    del test_data
except:
    pass

try:
    del val_data
except:
    pass

already_loaded = False
mimic4 = False
dataloader_train, dataloader_val, dataloader_test = LoadDataSets(batch_size=64,mimic4=mimic4)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

# Train a specific LSTM+CNN model

In [ ]:
randseed()

# Hyperparameter Grid Search

In [ ]:
# hyperparam simple grid search

lr_ = [1e-3, 1e-4, 1e-5]
hidden_dim_ = [8, 16, 128]
lstm_layers_ = [2, 3]
dropout_ = [0.2, 0.3, 0.5]
dropout_w_ = [0.2, 0.3, 0.5]
dropout_conv_ = [0.2, 0.3, 0.5]

save_results = []

for lr in lr_:
    number_epochs = 2*int(np.exp(np.log(1/lr)/np.log(2))/(2/lr))
    for hidden_dim in tqdm(hidden_dim_):
        for lstm_layers in tqdm(lstm_layers_):
            for dropout in tqdm(dropout_):
                for dropout_w in tqdm(dropout_w_):
                    for dropout_conv in tqdm(dropout_conv_):
                        (best_loss, best_accuracy, best_roc_auc), train_loss, val_loss, modelsignature = trainer(
                                 dataloader_train, dataloader_val, number_epochs=number_epochs,
                                 hidden_dim=hidden_dim, lstm_layers=lstm_layers,lr=lr,
                                 dropout=dropout, dropout_w=dropout_w, dropout_conv=dropout_conv,
                                 best_loss=1.0, best_accuracy=0.0, best_roc_auc=0.5,
                                 early_stopping=5,
                                 verbatim=False)
                        #modelsignature = f"{number_epochs}_{hidden_dim}_{lstm_layers}_{lr}_{dropout}-{dropout_w}-{dropout_conv}"
                        save_results.append({modelsignature:[hidden_dim, lstm_layers, lr, dropout, dropout_w, dropout_conv, (best_loss, best_accuracy, best_roc_auc), train_loss, val_loss]})
                        print(f"*** number_epochs={number_epochs},hidden_dim={hidden_dim}, lstm_layers={lstm_layers},lr={lr},dropout={dropout}, dropout_w={dropout_w}, dropout_conv={dropout_conv}, best_loss={best_loss}, best_accuracy={best_accuracy}, best_roc_auc={best_roc_auc}")

